In [1]:
import add_signals
import numpy as np
import get_signal_weights
import polars as pl
import sf_quant.backtester as sfb
import sf_quant.performance as sfp
import seaborn as sns
import datetime as dt

/home/kylecm11/sf-operations/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-30 15:41:06,029	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
df = pl.read_parquet('../data/russell_3000_daily.parquet')

df = (
    df.sort('barrid', 'date').filter(pl.col('price').gt(5))
)

In [3]:
signals = add_signals.add_signals(df)

In [4]:
returns = signals.with_columns(pl.col('return').shift(-1).alias('fwd_return')).filter(
        (pl.col('date') == dt.date.fromisoformat('2013-06-27')) &
        (pl.col('momentum_alpha').is_not_null())
        ).select(['date', 'barrid', 'momentum_alpha', 'predicted_beta', 'fwd_return'
        ])

returns

date,barrid,momentum_alpha,predicted_beta,fwd_return
date,str,f64,f64,f64
2013-06-27,"""USA0C11""",0.019559,1.686404,-0.005902
2013-06-27,"""USA0SY1""",0.007381,1.502102,0.006311
2013-06-27,"""USA1151""",-0.014219,1.304527,-0.015138
2013-06-27,"""USA11I1""",0.000356,1.157299,-0.008055
2013-06-27,"""USA11J1""",-0.045062,1.499607,-0.021538
…,…,…,…,…
2013-06-27,"""USAZW01""",0.012783,1.364199,-0.006078
2013-06-27,"""USAZW11""",0.016191,1.427974,-0.004873
2013-06-27,"""USAZWD1""",-0.005738,1.305923,-0.005826


In [8]:
domain = np.logspace(-1, 2, 5)
tracking_error = []
for gamma in domain:
    print(gamma)
    gamma_weights = get_signal_weights.get_signal_weights(returns.lazy(), 'momentum', dt.date.fromisoformat('2013-06-27'), dt.date.fromisoformat('2013-06-27'), gamma=gamma)
    gamma_returns = get_signal_weights.get_returns_from_weights(gamma_weights)
    gamma_table = sfp.generate_summary_table(gamma_returns)
    print(gamma_table)
    tracking_error.append(gamma_table.item("Active", "Volatility (%)"))

2025-10-30 15:59:36,433	INFO worker.py:1851 -- Calling ray.init() again after it has already been called.


0.1


RayTaskError(FileNotFoundError): [36mray::_construct_portfolio()[39m (pid=3656282, ip=128.187.49.40)
  File "/home/kylecm11/sf-operations/.venv/lib/python3.13/site-packages/sf_quant/backtester/parallel.py", line 30, in _construct_portfolio
    construct_covariance_matrix(date_, barrids).drop("barrid").to_numpy()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^
  File "/home/kylecm11/sf-operations/.venv/lib/python3.13/site-packages/sf_quant/data/covariance_matrix.py", line 63, in construct_covariance_matrix
    _construct_factor_exposure_matrix(date_, barrids).drop("barrid").to_numpy()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^
  File "/home/kylecm11/sf-operations/.venv/lib/python3.13/site-packages/sf_quant/data/covariance_matrix.py", line 95, in _construct_factor_exposure_matrix
    load_exposures_by_date(date_)
    ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^
  File "/home/kylecm11/sf-operations/.venv/lib/python3.13/site-packages/sf_quant/data/exposures.py", line 94, in load_exposures_by